<a href="https://colab.research.google.com/github/Himanshu0518/NLP/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:01<00:00, 90.0MB/s]
100% 80.9M/80.9M [00:01<00:00, 76.8MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1')

In [ ]:
df.columns = ['sentiments','id','date','query','user','text']

In [ ]:
df.head()

,sentiments,id,date,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
df.tail()

,sentiments,id,date,query,user,text
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
df.shape

(1599999, 6)



*   0 -- > For negative reviews
*   4 --> For positive reviews



Checking for Missing values

In [ ]:
df.isnull().sum()

,0
sentiments,0
id,0
date,0
query,0
user,0
text,0


In [ ]:
df['query'].value_counts()

,count
query,
NO_QUERY,1599999


In [ ]:
df = df.drop(['id','date','query','user'] , axis = 1)

In [ ]:
sum(df['text'].apply(len)<5)

0

In [ ]:
df = df[df['text'].apply(len)>5]
df.shape

(1599999, 2)

In [ ]:
df.sample(10)

,sentiments,text
1585461,4,"@crissangel I can't wait 4 the new season, I'm..."
437666,0,Someone pointed out in the bugs list @precentr...
1573059,4,@JoshFittell awe ima for sure come get him lol
224393,0,"he is sick, now i'm getting worry about him"
698464,0,today is dragging nearly time for work now
316284,0,Is sad and now must go back to sleep
18102,0,because im having a miserable lonely night
402897,0,"Ok, hearing the rain outside is making me home..."
165603,0,"So now, I have conjunctivitis in my left eye. ..."
1153835,4,@kyza it's pretty good! (especially compared t...


In [ ]:
df.head()

,sentiments,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [ ]:
df.sentiments.value_counts()

,count
sentiments,
4,800000
0,799999


Replace the positive reviews by 1

In [ ]:
df['sentiments'] = df['sentiments'].replace({4:1})

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords.words('english')[0:5]

['i', 'me', 'my', 'myself', 'we']

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
stemming('@Kenichan I dived many times for the ball. Man...')

'kenichan dive mani time ball man'

In [ ]:
df['text'] = df['text'].apply(stemming)

In [ ]:
df.head()

,sentiments,text
0,0,upset updat facebook text might cri result sch...
1,0,kenichan dive mani time ball manag save rest g...
2,0,whole bodi feel itchi like fire
3,0,nationwideclass behav mad see
4,0,kwesidei whole crew


In [ ]:
X = df['text'].values
Y = df['sentiments'].values

splitting the data into train and text data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.001,random_state = 2)

In [ ]:
X_train.shape , X_test.shape

((1598399,), (1600,))

# TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)  # Apply on raw text data
X_test_tfidf = vectorizer.transform(X_test)        # Apply on raw text data


In [ ]:
vocab = vectorizer.get_feature_names_out()
print("Vocabalory: ",vocab)

Vocabalory:  ['aa' 'aaaaaag' 'aaaaw' ... 'zm' 'zuckerbabi' 'zuzufalta']


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train_tfidf,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred = classifier.predict(X_test_tfidf)
accuracy_score(Y_test,y_pred)

0.7625

# LSTM and tensorflow


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
vocab_size = 10000

# One Hot Representation

In [ ]:
message = X.copy()

In [ ]:
message.size

1599999

In [ ]:
onehot_rep = [one_hot(words,vocab_size) for words in message]
onehot_rep[0:5]

[[7193, 6713, 8450, 8658, 3319, 7703, 2521, 4789, 2122, 4240, 8826],
 [7869, 870, 3154, 2343, 3623, 7020, 7933, 8025, 7555, 8427],
 [9794, 3602, 4363, 4884, 6286, 8749],
 [9573, 9310, 5905, 2975],
 [1879, 9794, 676]]

In [ ]:
print(message[0])
onehot_rep[0]

upset updat facebook text might cri result school today also blah


[7193, 6713, 8450, 8658, 3319, 7703, 2521, 4789, 2122, 4240, 8826]

# Embedding representation

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)

In [ ]:
print(embedded_docs[0])
print(type(embedded_docs[0]))

[   0    0    0    0    0    0    0    0    0 7193 6713 8450 8658 3319
 7703 2521 4789 2122 4240 8826]
<class 'numpy.ndarray'>


In [ ]:
# Creating the model
embedding_vector_features = 50
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(150))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer = 'adam' , metrics =['accuracy'])

# Build the model by specifying the input shape
model.build((None, sent_length))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 20, 50)              │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 20, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 150)                 │         120,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             151 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 620,751 (2.37 MB)

 Trainable params: 620,751 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [ ]:
Y_final

array([0, 0, 2, ..., 1, 1, 2])

In [ ]:
X_final.shape , Y_final.shape

((4869, 20), (4869,))

In [ ]:
train_data,test_data,train_label,test_label = train_test_split(X_final,Y_final,test_size=0.2,random_state = 4)

In [ ]:
model.fit(train_data,train_label,validation_data=(test_data,test_label),epochs=7,batch_size=64)

Epoch 1/7
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1210s 60ms/step - accuracy: 0.7438 - loss: 0.5138 - val_accuracy: 0.7647 - val_loss: 0.4834
Epoch 2/7
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1215s 60ms/step - accuracy: 0.7705 - loss: 0.4755 - val_accuracy: 0.7699 - val_loss: 0.4757
Epoch 3/7
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1346s 67ms/step - accuracy: 0.7785 - loss: 0.4624 - val_accuracy: 0.7723 - val_loss: 0.4713
Epoch 4/7
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1264s 60ms/step - accuracy: 0.7837 - loss: 0.4541 - val_accuracy: 0.7734 - val_loss: 0.4698
Epoch 5/7
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1213s 61ms/step - accuracy: 0.7887 - loss: 0.4462 - val_accuracy: 0.7732 - val_loss: 0.4721
Epoch 6/7
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1189s 59ms/step - accuracy: 0.7922 - loss: 0.4406 - val_accuracy: 0.7738 - val_loss: 0.4704
Epoch 7/7
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 1250s 62ms/step - accuracy: 0.7946 - loss: 0.4360 - val_accuracy: 0.7731 - val_loss: 0.4731


In [ ]:
print(Y_final)

[0 0 0 ... 1 1 1]


In [ ]:
y_pred = model.predict(test_data)

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 134s 13ms/step


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
y_pred = np.where(y_pred >= 0.5,1,0)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
confusion_matrix(test_label,y_pred)

array([[124016,  36043],
       [ 36579, 123362]])

In [ ]:
accuracy_score(y_pred,test_label)

0.77305625

In [ ]:
import pickle

In [ ]:
filename = 'sentiment_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model = pickle.load(open('sentiment_model.sav','rb'))

let us check how our loaded model works


In [ ]:
Y_pred = loaded_model.predict(test_data)

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 145s 14ms/step


In [ ]:
model.save('sentiment_model.h5')

In [ ]:
config = {
    'vocab_size': vocab_size,
    'sent_length': sent_length,
    'word_index': {}  # If using a specific word index, save it here
}

In [ ]:
with open('preprocessing_config.pkl', 'wb') as f:
    pickle.dump(config, f)